<a href="https://colab.research.google.com/github/aj1365/Machine-Learning-Models/blob/main/3D_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import required packages**

In [ ]:
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization,GlobalAveragePooling2D
from keras.layers import Dropout, Input
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

from plotly.offline import init_notebook_mode

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral

init_notebook_mode(connected=True)
%matplotlib inline

# **You can use GPU**


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# **Load your data**

In [ ]:
def loadData(name):
    
    data_path = os.path.join(os.getcwd(),'E:/PolSAR/')
   
    if name == 'Flevoland':
        
        data = sio.loadmat(os.path.join(data_path, 'Flevoland.mat'))['Flevoland']
        labels = sio.loadmat(os.path.join(data_path, 'Flevoland_15cls.mat'))['label']
  
    if name == 'SanFrancisco':
        
        data = sio.loadmat(os.path.join(data_path, 'SanFrancisco.mat'))['SanFrancisco']
        labels = sio.loadmat(os.path.join(data_path, 'SanFrancisco_gt.mat'))['label']

    
    return data, labels

In [ ]:
## GLOBAL VARIABLES
dataset = 'SanFrancisco'
test_ratio = 0.90
windowSize = 12

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=8, removeZeroLabels = True):
    margin = int((windowSize) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin , c - margin:c + margin ]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

# **Read your data**

In [ ]:
X , Y = loadData(dataset)

In [ ]:
X1, Y1 = createImageCubes(X, Y, windowSize=windowSize)
X1.shape, Y1.shape

In [ ]:
X1 = X1.reshape((X1.shape[0],windowSize,windowSize,12,1))
#X=X[:,:,:,0:10]
X1.shape

In [ ]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X1, Y1, test_ratio)

In [ ]:
# nunber of classes 5 for San Francisco and 15 for Flevoland
output_units = 5

# **Build your 3D CNN model**

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

In [ ]:
def get_cnn_model():
 
    input_shape =  12, 12, 12,1

     
    imIn = Input(shape=input_shape)
    
    conv_layer1 = Conv3D(filters=32, kernel_size=(1, 1, 7), activation='relu', padding='same')(imIn)
    conv_layer2 = Conv3D(filters=64, kernel_size=(3, 3, 5), activation='relu',padding='same')(conv_layer1)
    conv_layer3 = Conv3D(filters=64, kernel_size=(5, 5, 7), activation='relu',padding='same')(conv_layer2)
    conv3d_shape = conv_layer3.shape
    conv_layer3 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer3)
    conv_layer4 = Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same')(conv_layer3)
    conv_layer5 = Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding='same')(conv_layer4)
    conv_layer5 = GlobalAveragePooling2D()(conv_layer5)
    
    flatten_layer = Flatten()(conv_layer5)
    dense_layer1 = Dense(units=100, activation='relu')(flatten_layer)
    dense_layer2 = Dense(units=50, activation='relu')(dense_layer1)
    dense_layer1 = Dropout(0.5)(dense_layer2)
  
    output_layer = Dense(units=output_units, activation='softmax')(dense_layer1)
 
    model = Model(inputs=[imIn], outputs=output_layer)
    model.summary()
    plot_model(model, to_file='E:/PolSAR/3DClassifier.png', show_shapes=True, show_layer_names=True)
    

    return model

In [ ]:
model = get_cnn_model()

# **Intialize your 3D CNN model**

In [ ]:
weight_decay = 0.0001
batch_size = 32
dropout_rate = 0.5
learning_rate = 0.001

In [ ]:
import tensorflow_addons as tfa

# **Train your 3D CNN model**

In [ ]:
optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

checkpoint_filepath = "E:/PolSAR/CNN_SanF"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

history = model.fit(
        x=Xtrain,
        y=ytrain,
        batch_size=batch_size,
        epochs=100,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

In [ ]:
plt.figure(figsize=(7,7)) 
plt.grid() 
plt.plot(history.history['loss'])

plt.savefig('E:/PolSAR/3DClassifier.tiff',facecolor='w', dpi=500)

# **Test your 3D CNN model**

In [ ]:
Xtest = Xtest.reshape(-1, 12, 12, 12,1)


Xtest.shape

In [ ]:
Y_pred_test = model.predict([Xtest])
y_pred_test = np.argmax(Y_pred_test, axis=1)


In [ ]:
ca = np.sum(y_pred_test == ytest) / ytest.shape[0]

print("Classification accuracy: %.5f" % ca)

In [ ]:
# Classification Report
print(classification_report(ytest, y_pred_test,
      target_names = [
          
 'Bare Soil',
 'Mountain',
 'Water',
 'Building',
 'Vegetation',


                      ]))